In [1]:
import os
import cv2
import numpy as np
import glob
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
import seaborn as sns 
import tensorflow as tf
import tensorflow 
from tensorflow import keras
import segmentation_models as sm
from segmentation_models import Unet
import glob
from patchify import patchify

Segmentation Models: using `keras` framework.


In [2]:
root_directory= "rs19_val/"

In [3]:
patch_size=256

In [6]:
image_dataset = []  
for path, subdirs, files in os.walk(root_directory):
    print(path)  

rs19_val/
rs19_val/jpgs
rs19_val/jpgs\rs19_val
rs19_val/jsons
rs19_val/jsons\rs19_val
rs19_val/uint8
rs19_val/uint8\rs19_val


In [10]:
from PIL import Image

In [11]:

#Capture training image info as a list
image_dataset = []

for directory_path in glob.glob('rs19_val/jpgs/rs19_val/'):
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        #print(img_path)
        image = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        SIZE_X = (image.shape[1]//patch_size)*patch_size #Nearest size divisible by our patch size
        SIZE_Y = (image.shape[0]//patch_size)*patch_size #Nearest size divisible by our patch size
        image = Image.fromarray(image)
        image = image.crop((0 ,0, SIZE_X, SIZE_Y))  #Crop from top left corner
        #image = image.resize((SIZE_X, SIZE_Y))  #Try not to resize for semantic segmentation
        image = np.array(image) 
        #train_labels.append(label)
#Convert list to array for machine learning processing        


In [12]:
#Extract patches from each image
print("Now patchifying image:", img_path)
patches_img = patchify(image, (patch_size, patch_size, 3), step=patch_size)  #Step=256 for 256 patches means no overlap

Now patchifying image: rs19_val/jpgs/rs19_val\rs08499.jpg


In [14]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = MinMaxScaler()

In [15]:
for i in range(patches_img.shape[0]):
                    for j in range(patches_img.shape[1]):
                        
                        single_patch_img = patches_img[i,j,:,:]
                        
                        #Use minmaxscaler instead of just dividing by 255. 
                        single_patch_img = scaler.fit_transform(single_patch_img.reshape(-1, single_patch_img.shape[-1])).reshape(single_patch_img.shape)
                        
                        #single_patch_img = (single_patch_img.astype('float32')) / 255. 
                        single_patch_img = single_patch_img[0] #Drop the extra unecessary dimension that patchify adds.                               
                        image_dataset.append(single_patch_img)

In [18]:
#Capture mask/label info as a list
masks_dataset = [] 
for directory_path in glob.glob('rs19_val/uint8/rs19_val/'):
    for mask_path in glob.glob(os.path.join(directory_path, "*.png")):
        mask = cv2.imread(mask_path, 0)       
        SIZE_X = (mask.shape[1]//patch_size)*patch_size #Nearest size divisible by our patch size
        SIZE_Y = (mask.shape[0]//patch_size)*patch_size #Nearest size divisible by our patch size
        mask = Image.fromarray(mask)
        mask = mask.crop((0 ,0, SIZE_X, SIZE_Y))  #Crop from top left corner
        mask = np.array(mask) 

In [23]:
#Extract patches from each image
print("Now patchifying mask:", mask_path)
patches_mask = patchify(mask, (patch_size, patch_size, 3), step=patch_size)  #Step=256 for 256 patches means no overlap

Now patchifying mask: rs19_val/uint8/rs19_val\rs08499.png


ValueError: `window_shape` is incompatible with `arr_in.shape`

In [22]:
for i in range(patches_mask.shape[0]):
    for j in range(patches_mask.shape[1]):
        single_patch_mask = patches_mask[i,j,:,:]
        #single_patch_img = (single_patch_img.astype('float32')) / 255. #No need to scale masks, but you can do it if you want
        single_patch_mask = single_patch_mask[0] #Drop the extra unecessary dimension that patchify adds.                               
        masks_dataset.append(single_patch_mask) 

NameError: name 'patches_mask' is not defined